<a href="https://colab.research.google.com/github/nayan130/Nayans-Project/blob/master/pc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

Preliminary steps to compile and install the application:

In [1]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/SerenaC94/rte-rrtmgp-cpp.git

Cloning into 'rte-rrtmgp-cpp'...
remote: Enumerating objects: 7394, done.
remote: Counting objects: 100% (2147/2147), done.
remote: Compressing objects: 100% (390/390), done.
remote: Total 7394 (delta 1825), reused 2031 (delta 1755), pack-reused 5247
Receiving objects: 100% (7394/7394), 6.65 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (5660/5660), done.


In [3]:
%cd rte-rrtmgp-cpp/

/content/rte-rrtmgp-cpp


In [4]:
!git checkout cuda

Branch 'cuda' set up to track remote branch 'cuda' from 'origin'.
Switched to a new branch 'cuda'


In [5]:
!git submodule update --init

Submodule 'rte-rrtmgp' (https://github.com/earth-system-radiation/rte-rrtmgp) registered for path 'rte-rrtmgp'
Cloning into '/content/rte-rrtmgp-cpp/rte-rrtmgp'...
Submodule path 'rte-rrtmgp': checked out 'e9b7cbd70f286e6152e69f52c1a80018a13913b8'


In [6]:
!mkdir build

In [7]:
%cd build

/content/rte-rrtmgp-cpp/build


In [8]:
!cmake .. -DSYST=ubuntu -DUSECUDA=on

-- Precision: Double (64-bits floats)
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- The Fortran compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 11.8.89
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/gcc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting Fortran compiler ABI info
-- Detecting Fortran compiler ABI info - done
-- Check for working Fortran compiler: /usr/bin/gfortran - skipped
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting

In [9]:
!make

[  2%] Building Fortran object src_kernels/CMakeFiles/rte_rrtmgp_kernels.dir/mo_rte_kind.F90.o
[  5%] Building Fortran object src_kernels/CMakeFiles/rte_rrtmgp_kernels.dir/__/rte-rrtmgp/rte/mo_rte_util_array.F90.o
[  7%] Building Fortran object src_kernels/CMakeFiles/rte_rrtmgp_kernels.dir/__/rte-rrtmgp/rrtmgp/kernels/mo_gas_optics_kernels.F90.o
[ 10%] Building Fortran object src_kernels/CMakeFiles/rte_rrtmgp_kernels.dir/__/rte-rrtmgp/rrtmgp/kernels/mo_rrtmgp_util_reorder_kernels.F90.o
[ 12%] Building Fortran object src_kernels/CMakeFiles/rte_rrtmgp_kernels.dir/__/rte-rrtmgp/rte/kernels/mo_fluxes_broadband_kernels.F90.o
[ 15%] Building Fortran object src_kernels/CMakeFiles/rte_rrtmgp_kernels.dir/__/rte-rrtmgp/rte/kernels/mo_optical_props_kernels.F90.o
[ 17%] Building Fortran object src_kernels/CMakeFiles/rte_rrtmgp_kernels.dir/mo_fluxes_byband_kernels.F90.o
[ 20%] Building Fortran object src_kernels/CMakeFiles/rte_rrtmgp_kernels.dir/mo_rte_solver_kernels.F90.o
[ 22%] Linking Fortran st

### **Example**

Run the application using helper scripts:

In [10]:
%cd ../allsky

/content/rte-rrtmgp-cpp/allsky


In [11]:
!./make_links.sh

In [12]:
!python allsky_init.py

In [13]:
!python allsky_run.py

###### Starting RTE+RRTMGP solver ######
Solver settings:
cloud-optics         = true
fluxes               = true
longwave             = true
output-bnd-fluxes    = false
output-optical       = false
shortwave            = true
Reading atmospheric input data from NetCDF.
Preparing NetCDF output file.
Initializing the longwave solver.
Solving the longwave radiation.
Duration longwave solver: 89.612812 (ms)
Storing the longwave output.
Initializing the shortwave solver.
Solving the shortwave radiation.
Duration shortwave solver: 82.481817 (ms)
Storing the shortwave output.
###### Finished RTE+RRTMGP solver ######


In [14]:
!python compare-to-reference.py

Variable lw_flux_up differs (max abs difference: 1.976355e-07; max percent difference: 6.903320e-08%)
Variable lw_flux_dn differs (max abs difference: 7.972050e-07; max percent difference: 3.308960e-07%)
Variable sw_flux_up differs (max abs difference: 1.065513e-07; max percent difference: 1.109829e-07%)
Variable sw_flux_dn differs (max abs difference: 1.396712e-07; max percent difference: 1.586108e-08%)
Variable sw_flux_dir differs (max abs difference: 1.676650e-07; max percent difference: 2.027790e-08%)


### **Challenge**

Your task is now to translate into serial C code the `gas_optical_depths_minor` kernel, which can be found at /content/rte-rrtmgp-cpp/src_kernels_cuda/gas_optics_kernels.cu. The kernel is launched twice in the `compute_tau_absorption` function in /content/rte-rrtmgp-cpp/src_kernels_cuda/gas_optics_kernel_launchers.cu

One point is given to all those who succesfully **a) translate the kernel, b) verify that the accuracy of the application did not drop significantly, and c) measure the performance loss**.

One additional point will be given to those who also find a way to **improve the performance** of the application with respect to the serial C baseline **(going back to the original CUDA implementation does not count)**.

### **Final task list**


1. Upload on WeBeep a short report (~2 pages) explaining what you did and showing the results you obtained by the deadline.

2. Create a branch with a unique name (nickname/favourite actor/whatever, just not something that another of your classmates would use) and push your version of the code to the GitHub repository after the deadline (to avoid plagiarism as much as possible). **NB:** all branches with `cuda` in their name are locked to avoid overwriting the one that stands as a starting point for everyone.

3. Be prepared to present what you did in class (no need for a powerpoint, just speak and show your code).

### **Additional information**

If you are curious about what this application does and where it comes from, here are some useful resources:

*   https://en.wikipedia.org/wiki/Weather_Research_and_Forecasting_Model
*   https://github.com/earth-system-radiation/rte-rrtmgp
*   https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2019MS001621


